# Import Packages 

In [1]:
import pandas as pd
import numpy as np
import regex as re
pd.set_option('display.max_columns', 50)

# Load Data

In [2]:
path0 = "../Datasets/fema_ll_ss_houston.csv"
ss_df = pd.read_csv(path0)
ss_df.shape

(425, 25)

In [3]:
path1 = "../Datasets/fema_ll_fws_houston.csv"
fws_df = pd.read_csv(path1)
fws_df.shape

(2179, 25)

In [4]:
path2 = "../Datasets/fema_ll_fws2_houston.csv"
fws2_df = pd.read_csv(path2)
fws2_df.shape

(5200, 25)

In [5]:
path3 = "../Datasets/fema_ll_hm_houston.csv"
hm_df = pd.read_csv(path3)
hm_df.shape

(1914, 25)

In [6]:
path4 = "../Datasets/fema_ll_en_houston.csv"
en_df = pd.read_csv(path4)
en_df.shape

(189, 25)

In [7]:
path5 = "../Datasets/fema_ll_coms_houston.csv"
coms_df = pd.read_csv(path5)
coms_df.shape

(1361, 25)

In [8]:
path6 = "../Datasets/fema_ll_trans_houston.csv"
trans_df = pd.read_csv(path6)
trans_df.shape

(450, 25)

In [9]:
path7 = "../Datasets/fema_ll_haz_houston.csv"
haz_df = pd.read_csv(path7)
haz_df.shape

(1060, 25)

# Combining Dataframes

In [10]:
# Iterate over dataframes and assign Lifeline number according to list sequence.
df_list = [ss_df, fws_df, hm_df, en_df, coms_df, trans_df, haz_df, fws2_df]

for i, df in enumerate(df_list): 
    df['fema_l1'] = int(i + 1)

In [11]:
# Combine the dataframes together
df = ss_df.append(other = fws_df)
df = df.append(other = hm_df)
df = df.append(other = en_df)
df = df.append(other = coms_df)
df = df.append(other = trans_df)
df = df.append(other = haz_df)
# food, water, shelter overflow dataframe
df = df.append(other = fws2_df)

In [12]:
df.shape

(12778, 25)

In [13]:
# Map lifeline 2 (food & shelter) over the category overflow value, 8
lifeline_dict =  {1:int(1), 2:int(2), 3:int(3), 4:int(4), 5:int(5), 6:int(6), 7:int(7), 8:int(2)}

df['fema_l1'] = df['fema_l1'].map(lifeline_dict)

In [14]:
# Save process so far
path = "./combined_df_houston.csv"
data_frame = df.to_csv(path, index = False)

###  Other datacleaning

### Summary:

- Reset index and dropped the extra index column
- It doesn't seem that there are any duplicates, though I am slightly skeptical of this because I know Yelp pulls from nearby cities. 
- Categories column cleaned up
- When looking at the null values, we decided to drop the display phone, phone and image_url columns. 
- We also decided to drop the distance and transactions columns
- We also combined address columns 1, 2 and 3 into one. 
- 68 missing zipcodes were filled in by plugging in the latitudes and logitudes using https://developers.google.com/maps/documentation/javascript/examples/geocoding-reverse
- Price null values are going to be kept for now.

In [15]:
# Re-order columns to a better sequence
subset = ['id', 'name', 'fema_l1', 'categories', 'latitude', 'longitude', 'review_count', 'rating', 'price', 
          'zip_code', 'city', 'state', 'country', 'display_address', 'display_phone', 'alias', 'phone','address1', 
          'address2', 'address3','is_closed','url','distance', 'image_url', 'transactions']

df = df[subset]

In [17]:
# Sort dataframe by 'id' and reset the index to this sequence
df.sort_values(by='id', inplace = True)
df.reset_index(inplace=True)
# Drop duplicate rows with the same ID and Fema Lifeline number
df.drop_duplicates(subset=['id', 'fema_l1'], keep = "first", inplace=True)
df.shape

# Drop the old index column
df.drop(columns="index", inplace = True)

In [18]:
# Clean up the string format in 'categories' series 
df["categories"] = df["categories"].map(lambda string: string.replace("[", ""))
df["categories"] = df["categories"].map(lambda string: string.replace("]", ""))
df["categories"] = df["categories"].map(lambda string: string.replace("'",""))
df["categories"] = df["categories"].map(lambda string: string.split(',')[0])

In [19]:
# Define a function to map through the listing 'name' and return the lifeline number
def get_all_lifelines(name):
    subset = df[df['name'] == name]
    lifelines = list(set(subset['fema_l1']))
    return lifelines

In [20]:
# Create a dictionary of the listing name and its list of all lifeline numbers
all_names_lifelines = {}

for name in list(set(df['name'])):
    all_names_lifelines[name] = get_all_lifelines(name)

# Create a new 'all lifelines' series and use the function above to populate lifeline numbers 
df['all_lifelines'] = df['name'].map(all_names_lifelines)

In [22]:
# Manually create dummy columns for each lifeline 
df['lifeline_1'] = 0
df['lifeline_2'] = 0
df['lifeline_3'] = 0
df['lifeline_4'] = 0
df['lifeline_5'] = 0
df['lifeline_6'] = 0
df['lifeline_7'] = 0

In [23]:
# Iterate over each row in 'all_lifelines' and populate 1 if in the sequence
for ix, each in enumerate(df['all_lifelines']):
    if 1 in each:
        df.loc[ix, 'lifeline_1'] = int(1)
    if 2 in each:
        df.loc[ix, 'lifeline_2'] = 1
    if 3 in each:
        df.loc[ix, 'lifeline_3'] = 1
    if 4 in each:
        df.loc[ix, 'lifeline_4'] = 1
    if 5 in each:
        df.loc[ix, 'lifeline_5'] = 1
    if 6 in each:
        df.loc[ix, 'lifeline_6'] = 1
    if 7 in each:
        df.loc[ix, 'lifeline_7'] = 1

df.drop(columns=['fema_l1', 'all_lifelines'], inplace=True)

In [24]:
# Drop unneccessary columns
df.drop(columns=["distance", "phone", "image_url", 'transactions' ], inplace = True)

In [25]:
# Drop duplicates according to ID now that all Lifeline number are accounted for in each row
df.drop_duplicates(subset=['id'], inplace = True)

In [26]:
df.isnull().sum()

id                    1
name                  1
categories            1
latitude             12
longitude            12
review_count          1
rating                1
price                 1
zip_code             28
city                  1
state                 1
country               1
display_address       1
display_phone       269
alias                 1
address1            217
address2           5417
address3           6964
is_closed             1
url                   1
lifeline_1            1
lifeline_2            0
lifeline_3            1
lifeline_4            1
lifeline_5            1
lifeline_6            1
lifeline_7            1
dtype: int64

In [27]:
# Fill null values in the address series and concatenate them into one column
df["address1"].fillna(" ", inplace=True)
df["address2"].fillna(" ", inplace=True)
df["address3"].fillna(" ", inplace=True)
df["address"] = df["address1"].str.cat(df["address2"], sep= " ")
df["address"] = df["address"].str.cat(df["address3"], sep= " ")

# Drop the address columns
df.drop(columns=["address1", "address2", "address3"], inplace=True)

# Fill null phone listings
df["display_phone"].fillna("000-000-000", inplace=True)

# Fill null 'price' cells
df["price"] = df["price"].map(lambda x: np.nan if x == "NAN" else x)

# Drop rows where 'latitude' and 'longitude' series are null 
df.dropna(axis = 0, subset=["latitude", "longitude"], inplace = True)

In [28]:
df.isnull().sum()

id                    0
name                  0
categories            0
latitude              0
longitude             0
review_count          0
rating                0
price              3177
zip_code             16
city                  0
state                 0
country               0
display_address       0
display_phone         0
alias                 0
is_closed             0
url                   0
lifeline_1            0
lifeline_2            0
lifeline_3            0
lifeline_4            0
lifeline_5            0
lifeline_6            0
lifeline_7            0
address               0
dtype: int64

In [29]:
df.shape

(7157, 25)

In [31]:
df.head()

,id,name,categories,latitude,longitude,review_count,rating,price,zip_code,city,state,country,display_address,display_phone,alias,is_closed,url,lifeline_1,lifeline_2,lifeline_3,lifeline_4,lifeline_5,lifeline_6,lifeline_7,address
0,--4m48NzUAHvTd4v0u5YIg,Christian's Tailgate Bar and Grill,sportsbars,29.752573,-95.376317,353.0,3.0,$$,77002,Houston,TX,US,"['2000 Bagby St', 'Houston, TX 77002']",(713) 527-0261,christians-tailgate-bar-and-grill-houston,False,https://www.yelp.com/biz/christians-tailgate-b...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2000 Bagby St
3,--4uMhyYMhURrd3dgwe_mA,Tu-Go Kitchen,foodtrucks,29.710531,-95.381760,7.0,2.0,$$,77021,Houston,TX,US,"['1 Hermann Park Ct', 'Houston, TX 77021']",(832) 910-7533,tu-go-kitchen-houston,False,https://www.yelp.com/biz/tu-go-kitchen-houston...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1 Hermann Park Ct
4,--tU4l_I9bKvJAeKYPbdSw,Treebeards,southern,29.755270,-95.363040,32.0,4.0,$,77010,Houston,TX,US,"['1200 McKinney St', 'Ste 329', 'Houston, TX 7...",(713) 400-9595,treebeards-houston-3,False,https://www.yelp.com/biz/treebeards-houston-3?...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1200 McKinney St Ste 329
6,-07hPjEFeAZVe7XSHIz54A,Hollywood Sign Hike,hiking,34.120281,-118.316141,178.0,4.5,NaN,90068,Los Angeles,CA,US,"['3200 Canyon Dr', 'Bronson Canyon Griffith Pa...",(323) 666-5046,hollywood-sign-hike-los-angeles,False,https://www.yelp.com/biz/hollywood-sign-hike-l...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3200 Canyon Dr Bronson Canyon Griffith Park
7,-1hWNeXT09Sjwbref49wrg,Chase Bank,banks,29.829029,-95.429360,6.0,2.5,NaN,77018,Houston,TX,US,"['4320 Ella Blvd', 'Houston, TX 77018']",(713) 680-4381,chase-bank-houston-174,False,https://www.yelp.com/biz/chase-bank-houston-17...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4320 Ella Blvd


In [32]:
path = "../Datasets/cleaned_dataframe_houston.csv"
data_frame = df.to_csv(path, index = False)